## Introduction
So far, you've learned how to use several SQL clauses. For instance, you know how to use SELECT to pull specific columns from a table, along with WHERE to pull rows that meet specified criteria. You also know how to use aggregate functions like COUNT(), along with GROUP BY to treat multiple rows as a single group.

Now you'll learn how to change the order of your results using the ORDER BY clause, and you'll explore a popular use case by applying ordering to dates. To illustrate what you'll learn in this tutorial, we'll work with a slightly modified version of our familiar pets table.

## ORDER BY
ORDER BY is usually the last clause in your query, and it sorts the results returned by the rest of your query.

Notice that the rows are not ordered by the ID column. We can quickly remedy this with the query below.

In [2]:
from pandas.io import gbq

In [9]:
df_pets_records = gbq.read_gbq('select ID,Name,Animal from pet_records.pet_records order by ID',
                               project_id = 'numeric-dialect-275303', index_col="ID")
df_pets_records

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.41rows/s]


,Name,Animal
ID,,
1,Dr. Harris Bonkers,Rabbit
2,Moon,Dog
3,Ripley,Cat
4,Tom,Cat


The ORDER BY clause also works for columns containing text, where the results show up in alphabetical order.

In [10]:
df_pets_records = gbq.read_gbq('select ID,Name,Animal from pet_records.pet_records order by Animal',
                               project_id = 'numeric-dialect-275303', index_col="ID")
df_pets_records

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.55rows/s]


,Name,Animal
ID,,
3,Ripley,Cat
4,Tom,Cat
2,Moon,Dog
1,Dr. Harris Bonkers,Rabbit


You can reverse the order using the DESC argument (short for 'descending'). The next query sorts the table by the Animal column, where the values that are last in alphabetic order are returned first.

In [11]:
df_pets_records = gbq.read_gbq('select ID,Name,Animal from pet_records.pet_records order by Animal DESC',
                               project_id = 'numeric-dialect-275303', index_col="ID")
df_pets_records

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.44rows/s]


,Name,Animal
ID,,
1,Dr. Harris Bonkers,Rabbit
2,Moon,Dog
3,Ripley,Cat
4,Tom,Cat


## Dates
Next, we'll talk about dates, because they come up very frequently in real-world databases. There are two ways that dates can be stored in BigQuery: as a DATE or as a DATETIME.

The DATE format has the year first, then the month, and then the day. It looks like this:

    YYYY-[M]M-[D]D
- YYYY: Four-digit year
- [M]M: One or two digit month
- [D]D: One or two digit day

So 2019-01-10 is interpreted as January 10, 2019.

The DATETIME format is like the date format ... but with time added at the end.

### EXTRACT
Often you'll want to look at part of a date, like the year or the day. You can do this with EXTRACT. We'll illustrate this with a slightly different table, called pets_with_date.

The query below returns two columns, where column Day contains the day corresponding to each entry the Date column from the pets_with_date table:

In [17]:
hacker_news = gbq.read_gbq('select * from hacker_news.hacker_news_dataset limit 100',
                           project_id = 'numeric-dialect-275303')
hacker_news.head()


Downloading: 100%|█████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 58.14rows/s]


,id,title,url,num_points,num_comments,author,created_at
0,12121216,Valid.ly Never send another OOPS message,https://www.valid.ly,1,1,validly,2016-07-19 12:05:00+00:00
1,11610310,Ask HN: Aby recent changes to CSS that broke m...,None,1,1,polskibus,2016-05-02 10:14:00+00:00
2,11590768,"Show HN: Shanhu.io, a programming playground p...",https://shanhu.io,1,1,h8liu,2016-04-28 18:05:00+00:00
3,10581844,"Analysis of 114 propaganda sources from ISIS, ...",http://37.252.122.95/sites/default/files/Insid...,1,1,crosre,2015-11-17 15:53:00+00:00
4,10402073,Predicting the Future and Exponential Growth,http://uday.io/2015/10/15/predicting-the-futur...,1,1,urs2102,2015-10-16 21:19:00+00:00


In [22]:
# kita coba menggunakan data hacker_news
hacker_news = gbq.read_gbq('select author,EXTRACT(Day from created_at) as Day from hacker_news.hacker_news_dataset limit 100',
                           project_id = 'numeric-dialect-275303')
hacker_news.head()

Downloading: 100%|█████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 44.23rows/s]


,author,Day
0,validly,19
1,polskibus,2
2,h8liu,28
3,crosre,17
4,urs2102,16


SQL is very smart about dates, and we can ask for information beyond just extracting part of the cell. For example, this query returns one column with just the week in the year (between 1 and 53) for each date in the Date column:

In [23]:
hacker_news = gbq.read_gbq('select author,EXTRACT(week from created_at) as Week from hacker_news.hacker_news_dataset limit 100',
                           project_id = 'numeric-dialect-275303')
hacker_news.head()

Downloading: 100%|█████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 62.77rows/s]


,author,Week
0,validly,29
1,polskibus,18
2,h8liu,17
3,crosre,46
4,urs2102,41


You can find all the functions you can use with dates in BigQuery in this documentation under "Date and time functions".
https://cloud.google.com/bigquery/docs/reference/legacy-sql#datetimefunctions

## Example: Which day of the week has the most fatal motor accidents?
Let's use the US Traffic Fatality Records database, which contains information on traffic accidents in the US where at least one person died.

We'll investigate the accident_2015 table. Here is a view of the first few rows. (We have hidden the corresponding code. To take a peek, click on the "Code" button below.)

Let's use the table to determine how the number of accidents varies with the day of the week. Since:

- the consecutive_number column contains a unique ID for each accident, and
- the timestamp_of_crash column contains the date of the accident in DATETIME format,

we can:

- EXTRACT the day of the week (as day_of_week in the query below) from the timestamp_of_crash column, and
- GROUP BY the day of the week, before we COUNT the consecutive_number column to determine the number of accidents for each day of the week.

Then we sort the table with an ORDER BY clause, so the days with the most accidents are returned first.

As usual, we run it as follows:

Notice that the data is sorted by the num_accidents column, where the days with more traffic accidents appear first.

To map the numbers returned for the day_of_week column to the actual day, you might consult the BigQuery documentation on the DAYOFWEEK function. It says that it returns "an integer between 1 (Sunday) and 7 (Saturday), inclusively". So, in 2015, most fatal motor accidents in the US occured on Sunday and Saturday, while the fewest happened on Tuesday.

## Your Turn
ORDER BY can make your results easier to interpret. Try it yourself.

# Exercise: Order By

## Introduction
You've built up your SQL skills enough that the remaining hands-on exercises will use different datasets than you see in the explanations. If you need to get to know a new dataset, you can run a couple of SELECT queries to extract and review the data you need.

The next exercises are also more challenging than what you've done so far. Don't worry, you are ready for it!

The World Bank has made tons of interesting education data available through BigQuery. Run the following cell to see the first few rows of the international_education table from the world_bank_intl_education dataset.

## Exercises

The value in the `indicator_code` column describes what type of data is shown in a given row.  

One interesting indicator code is `SE.XPD.TOTL.GD.ZS`, which corresponds to "Government expenditure on education as % of GDP (%)".

### 1) Government expenditure on education

Which countries spend the largest fraction of GDP on education?  

To answer this question, consider only the rows in the dataset corresponding to indicator code `SE.XPD.TOTL.GD.ZS`, and write a query that returns the average value in the `value` column for each country in the dataset between the years 2010-2017 (including 2010 and 2017 in the average). 

Requirements:
- Your results should have the country name rather than the country code. You will have one row for each country.
- The aggregate function for average is **AVG()**.  Use the name `avg_ed_spending_pct` for the column created by this aggregation.
- Order the results so the countries that spend the largest fraction of GDP on education show up first.

In case it's useful to see a sample query, here's a query you saw in the tutorial (using a different dataset):
```
# Query to find out the number of accidents for each day of the week
query = """
        SELECT COUNT(consecutive_number) AS num_accidents, 
               EXTRACT(DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """
```

##### Jawaban

### 2) Identify interesting codes to explore

The last question started by telling you to focus on rows with the code `SE.XPD.TOTL.GD.ZS`. But how would you find more interesting indicator codes to explore?

There are 1000s of codes in the dataset, so it would be time consuming to review them all. But many codes are available for only a few countries. When browsing the options for different codes, you might restrict yourself to codes that are reported by many countries.

Write a query below that selects the indicator code and indicator name for all codes with at least 175 rows in the year 2016.

Requirements:
- You should have one row for each indicator code.
- The columns in your results should be called `indicator_code`, `indicator_name`, and `num_rows`.
- Only select codes with 175 or more rows in the raw database (exactly 175 rows would be included).
- To get both the `indicator_code` and `indicator_name` in your resulting DataFrame, you need to include both in your **SELECT** statement (in addition to a **COUNT()** aggregation). This requires you to include both in your **GROUP BY** clause.
- Order from results most frequent to least frequent.

##### Jawaban

##### Coba-coba

In [35]:
hacker_news = gbq.read_gbq('SELECT num_points, num_comments, COUNT(1) AS num_rows FROM hacker_news.hacker_news_dataset WHERE author = "validly" GROUP BY num_points, num_comments HAVING COUNT(1) >= 10 ORDER BY COUNT(1) DESC',
                           project_id = 'numeric-dialect-275303')
hacker_news.head()


Downloading: 0rows [00:01, ?rows/s]


,num_points,num_comments,num_rows


Dataset berbeda, jadi query yang kita buat tidak memunculkan hasil yang kita mau karena tidak memiliki data yang sesuai query kita.